In [1]:
import os
from arguments import Args
import time
import logging
import numpy as np
import torch
from tqdm import tqdm
from core.raft_stereo import RAFTStereo, autocast
import core.stereo_datasets as datasets
from core.utils.utils import InputPadder
from PIL import Image

In [2]:
# get args
args = Args()
model = torch.nn.DataParallel(RAFTStereo(args), device_ids=[0])

# load model
if args.restore_ckpt is not None:
    assert args.restore_ckpt.endswith(".pth")
    logging.info("Loading checkpoint...")
    checkpoint = torch.load(args.restore_ckpt, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint, strict=True)
    logging.info(f"Done loading checkpoint")

if torch.cuda.is_available():
    model.cuda()
model.eval()

DataParallel(
  (module): RAFTStereo(
    (cnet): MultiBasicEncoder(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (relu1): ReLU(inplace=True)
      (layer1): Sequential(
        (0): ResidualBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (relu): ReLU(inplace=True)
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): ResidualBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (relu): ReLU(inplace=True)
          (norm1

# 1. pred disp

In [3]:
# set up paths
root_dir= "/home/cornelius/Documents/Bibliothek/ICL/IndependentStudyOption/scared_accurate_3ds"
save_path= "pred_disp/scared"
#options = MonodepthOptions()
#opt = opt.parse()
device = "cpu"

if os.path.exists(save_path)!=True:
    os.makedirs(save_path)

In [ ]:
def load_image(imfile):
    img = np.array(Image.open(imfile).convert('RGB')).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(device)


keyframes = 5
scared_d = [8, 9]
# run model on datasets 8 and 9
for i in (scared_d):
    for k in range(keyframes):
        # TODO: load data for eval!!!
        # Load image and preprocess
        image1 = load_image("{}/dataset{}_keyframe{}/Left_Image_rect.png".format(root_dir, i, k))
        image2 = load_image("{}/dataset{}_keyframe{}/Right_Image_rect.png".format(root_dir, i, k))
        
        padder = InputPadder(image1.shape, divis_by=32)
        image1, image2 = padder.pad(image1, image2)
        
        _, flow_up = model(image1, image2, iters=args.valid_iters, test_mode=True)
        plt.imshow(flow_up)
        plt.show()
        
        print('disparity output dataset {} keyframe {} complete!'.format(i, k))
        np.save('{}/dataset{}_keyframe{}_predicted_disparities.npy'.format(save_path, i, k), pred_disparities)

/home/cornelius/anaconda3/envs/deepLearn/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
